In [1]:
from torch_lib.Dataset import *
from torch_lib.Model import Model, OrientationLoss


import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision.models import vgg
from torch.utils import data

In [2]:
import os

In [3]:
train_path = os.path.join('dataset','20220428_qd3dt_1024')

In [4]:
dataset = Dataset(train_path)

In [5]:
attr = vars(dataset)

In [6]:
print(attr.keys())

dict_keys(['top_label_path', 'top_img_path', 'top_calib_path', 'proj_matrix', 'ids', 'num_images', 'bins', 'angle_bins', 'interval', 'overlap', 'bin_ranges', 'averages', 'object_list', 'labels', 'curr_id', 'curr_img'])


In [7]:
attr['averages'].get_item('Car')

array([0.63047619, 0.27809524, 0.24380952])

In [8]:
attr['proj_matrix']

array([[208.,   0., 512.,   0.],
       [  0., 208., 512.,   0.],
       [  0.,   0.,   1.,   0.]], dtype=float32)

In [9]:
attr['proj_matrix']

array([[208.,   0., 512.,   0.],
       [  0., 208., 512.,   0.],
       [  0.,   0.,   1.,   0.]], dtype=float32)

In [10]:
epochs = 100
batch_size = 64
alpha = 0.6
w = 0.4

In [11]:
params = {'batch_size': batch_size,
          'shuffle': True,
          'num_workers': 6}

In [12]:
generator = data.DataLoader(dataset, **params)

In [13]:
my_vgg = vgg.vgg19_bn(pretrained=True)
model = Model(features=my_vgg.features).cuda()
opt_SGD = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
conf_loss_func = nn.CrossEntropyLoss().cuda()
dim_loss_func = nn.MSELoss().cuda()
orient_loss_func = OrientationLoss

In [14]:
model_path = os.path.join('weights/')
latest_model = None
first_epoch = 0

In [15]:
total_num_batches = int(len(dataset) / batch_size)

In [16]:
total_num_batches

98

In [17]:
for epoch in range(first_epoch+1, epochs+1):
    curr_batch = 0
    passes = 0
    for local_batch, local_labels in generator:

        truth_orient = local_labels['Orientation'].float().cuda()
        truth_conf = local_labels['Confidence'].long().cuda()
        truth_dim = local_labels['Dimensions'].float().cuda()

        local_batch=local_batch.float().cuda()
        [orient, conf, dim] = model(local_batch)

        orient_loss = orient_loss_func(orient, truth_orient, truth_conf)
        dim_loss = dim_loss_func(dim, truth_dim)

        truth_conf = torch.max(truth_conf, dim=1)[1]
        conf_loss = conf_loss_func(conf, truth_conf)

        loss_theta = conf_loss + w * orient_loss
        loss = alpha * dim_loss + loss_theta

        opt_SGD.zero_grad()
        loss.backward()
        opt_SGD.step()


        if passes % 20 == 0:
            print("--- epoch %s | batch %s/%s --- [loss: %s]" %(epoch, curr_batch, total_num_batches, loss.item()))
            passes = 0

        passes += 1
        curr_batch += 1

    # save after every 10 epochs
    if epoch % 10 == 0:
        name = model_path + 'epoch_%s.pkl' % epoch
        print("====================")
        print ("Done with epoch %s!" % epoch)
        print ("Saving weights as %s ..." % name)
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': opt_SGD.state_dict(),
                'loss': loss
                }, name)
        print("====================")

--- epoch 1 | batch 0/98 --- [loss: 0.667971134185791]
--- epoch 1 | batch 20/98 --- [loss: 0.5073529481887817]
--- epoch 1 | batch 40/98 --- [loss: 0.4124372899532318]
--- epoch 1 | batch 60/98 --- [loss: 0.3846319913864136]
--- epoch 1 | batch 80/98 --- [loss: 0.3505670428276062]
--- epoch 2 | batch 0/98 --- [loss: 0.3280717134475708]
--- epoch 2 | batch 20/98 --- [loss: 0.22336970269680023]
--- epoch 2 | batch 40/98 --- [loss: 0.3210889995098114]
--- epoch 2 | batch 60/98 --- [loss: 0.2490725964307785]
--- epoch 2 | batch 80/98 --- [loss: 0.27617666125297546]
--- epoch 3 | batch 0/98 --- [loss: 0.19477112591266632]
--- epoch 3 | batch 20/98 --- [loss: 0.29385945200920105]
--- epoch 3 | batch 40/98 --- [loss: 0.33657386898994446]
--- epoch 3 | batch 60/98 --- [loss: 0.3348689377307892]
--- epoch 3 | batch 80/98 --- [loss: 0.2693672180175781]
--- epoch 4 | batch 0/98 --- [loss: 0.1744297444820404]
--- epoch 4 | batch 20/98 --- [loss: 0.3275681138038635]
--- epoch 4 | batch 40/98 --- [

Traceback (most recent call last):
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/alfin/.conda/envs/monodle/lib/python3.8/mult

KeyboardInterrupt: 